In [1]:
import os
import sys
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib as mpl
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt


In [2]:
###change the variables in this cell to what you need; shouldn't have to change anything else in the script
###For Ben: see "CPEX-CV Well Documented Convection" document for convective case time ranges

file_date = '20220922'          #which date to create clean DAWN data CSV for
start_time0 = '20220922,054000'  #start time (UTC) to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal
end_time0 = '20220922,083000'    #end time (UTC) to create clean DAWN data CSV for; gauge near-storm environment based on APR plots and IMERG/Hourly DC-8 track on CPEX Portal
day_folder = os.path.join(os.getcwd(), file_date)     #directory you want the QCed DAWN data to be saved to
dawn_folder = os.path.join(day_folder, 'DAWN_files')  #directory containing the given day's DAWN .nc file
dawn_final_name = os.path.join(day_folder, 'final_dawn_' + file_date + '.csv')  #filepath and filename that the QCed DAWN data will be saved to


In [3]:
start_time = datetime.strptime(start_time0, '%Y%m%d,%H%M%S')
end_time = datetime.strptime(end_time0, '%Y%m%d,%H%M%S')


In [4]:
test = xr.open_dataset('/Users/brodenkirch/Desktop/CPEX/Coding/20220906/DAWN_files/cpexcv-DAWN_DC8_20220906_R0.nc', group = '')
test


<xarray.Dataset>
Dimensions:                   (time: 1157, z: 434)
Coordinates:
  * time                      (time) datetime64[ns] 2022-09-06T11:09:29 ... 2...
    lat                       (time) float64 ...
    lon                       (time) float64 ...
  * z                         (z) float32 -0.015 0.015 0.045 ... 12.94 12.98
Data variables:
    Wind_Speed                (time, z) float32 ...
    Wind_Direction            (time, z) float32 ...
    Line_of_Sight_Wind_Speed  (time, z) float32 ...
    U_comp                    (time, z) float32 ...
    V_comp                    (time, z) float32 ...
    smoothed_U_comp           (time, z) float32 ...
    smoothed_V_comp           (time, z) float32 ...
    smoothed_Wind_Speed       (time, z) float32 ...
    smoothed_Wind_Direction   (time, z) float32 ...
Attributes: (12/35)
    Conventions:                  CF-1.10
    data_product_groups:          root
    data_processing_note:         none
    data_use_guideline:           For responsible scientific use of the data ...
    file_originator:              Kristopher Bedka (NASA Langley Research Cen...
    file_originator_contact:      Zhaoyan Liu (zhaoyan.liu@nasa.gov) and Kris...
    ...                           ...
    VersionID:                    RD
    summary:                      DAWN wind profiles collected from one of th...
    references:                   https://doi.org/10.5067/ASDC/SUBORBITAL/CPE...
    time_coverage_start:          2022-09-06T11:09:29.000000Z
    time_coverage_end:            2022-09-06T18:01:50.000000Z
    flight_start_date:            20220906

In [5]:
#test2 = xr.open_dataset('/Users/brodenkirch/Desktop/CPEX/Coding/20220906/DAWN_files/cpexcv-DAWN_DC8_20220906_R0.nc', group = 'Nav_Data')
#test2


In [6]:
#test3 = xr.open_dataset('/Users/brodenkirch/Desktop/CPEX/Coding/20220906/DAWN_files/cpexcv-DAWN_DC8_20220906_R0.nc', group = 'Data_Quality')
#test3


In [7]:
#test4 = xr.open_dataset('/Users/brodenkirch/Desktop/CPEX/Coding/20220906/DAWN_files/cpexcv-DAWN_DC8_20220906_R0.nc', group = 'Date_Time')
#test4


In [8]:
#loop through each DAWN file to filter/QC the data and add to the given date's final_dawn CSV

#merge the DAWN file's four NetCDF groups together into one dataset
group_names = ['', 'Nav_Data', 'Data_Quality', 'Date_Time']
datasets = []
for name in group_names:
    datasets.append(xr.open_dataset(os.path.join(dawn_folder, os.listdir(dawn_folder)[0]), group = name))  #only 1 DAWN file that contains all the DAWN profiles for the file_date)
ds = xr.merge(datasets)

first_file = True
for a in range(len(ds['time'])):
    profile = ds.isel(time = a)
    prof_time = pd.Timestamp(profile.time.values)#.strftime("%H%M%S")  #convert numpy.datetime64 to datetime object

    if (prof_time < start_time) or (prof_time > end_time):  #ignore DAWN profiles outside of the convective case's time range
        continue
    else:
        prof_df = profile.to_dataframe()
        #dawn_full_time = str(prof_df.time.iloc[-1])
        
        if prof_df.index[0] > 0:
            sys.exit('The first index is not negative....need to account for this in the script.')
        if prof_df.index[1] <= 0:
            sys.exit('The second index is not above the surface....need to account for this in the script.')
        
        #per the DAWN README, exclude the first two altitude bins above the surface
            #(so, omit -0.015, 0.015, and 0.045, which are the first 3 integer positions' heights (in km))
        prof_df = prof_df.iloc[3:].copy()
            #using .copy() to prevent chain-indexing --> https://www.dataquest.io/blog/settingwithcopywarning/
        
        #QC the data (per DAWN README): wind speed <= 50 m/s and >= 0, wind direction <= 360 and >= 0, 
                      #u/v magnitude <= 50 m/s, AC roll magnitude < 3, QC_flag != 1, z (in km, the index) > 0
        df_use = prof_df[(prof_df['Wind_Speed'] >= 0) & (prof_df['Wind_Speed'] <= 50) & 
                         (prof_df['Wind_Direction'] >= 0) & (prof_df['Wind_Direction'] <= 360) & 
                         (prof_df['U_comp'].abs() <= 50) & (prof_df['V_comp'].abs() <= 50) & 
                         (prof_df['AC_Roll'].abs() <= 3) & (prof_df['QC_flag'] != 1) & 
                         (prof_df.index > 0)].copy()
            #^^^ using .copy() to prevent chain-indexing --> https://www.dataquest.io/blog/settingwithcopywarning/
    
#         ##To create a CSV that includes NaN/blank wind speed/direction (to better plot pcolormesh plots of 
#             ##wind speed/direction, see CPEXCV_4panel_Giselle.ipynb), then QC the data using the 
#               #following 2 lines instead of the QC code above
#         df_use = prof_df[(prof_df['AC_Roll'].abs() <= 3) & (prof_df['QC_flag'] != 1) & 
#                          (prof_df.index > 0)].copy()
#             #^^^ using .copy() to prevent chain-indexing --> https://www.dataquest.io/blog/settingwithcopywarning/
        
        if len(df_use) > 0:  #i.e., good data exists for the profile
                    
            #grab the time of the first good data line (the last index, although all the times in 
                #the profile are the same), to be used as the official DAWN time
            dawn_full_time = str(df_use.time.iloc[-1])

            #convert the good, relevant DAWN data to a dataframe and add to the final DAWN CSV file
            df_use = df_use.iloc[::-1].copy()  #reverse the dataframe to order by decreasing height
            dawn_clean_df = pd.DataFrame(columns = ['Time [UTC]','Height [m]', 'Wind Direction [deg]', 'Wind Speed [m/s]', 'U Comp of Wind [m/s]', 'V Comp of Wind [m/s]', 'Latitude [deg]', 'Longitude [deg]'])
            dawn_clean_df['Time [UTC]'] = [dawn_full_time] * len(df_use)  #a list of len(df_use) with the same dawn_full_time value
            dawn_clean_df['Height [m]'] = np.round(list(df_use.index * 1000), 1)
            dawn_clean_df['Wind Direction [deg]'] = np.round(list(df_use['Wind_Direction']), 5)
            dawn_clean_df['Wind Speed [m/s]'] = np.round(list(df_use['Wind_Speed']), 5)
            dawn_clean_df['U Comp of Wind [m/s]'] = np.round(list(df_use['U_comp']), 5)
            dawn_clean_df['V Comp of Wind [m/s]'] = np.round(list(df_use['V_comp']), 5)
            dawn_clean_df['Latitude [deg]'] = np.round(list(df_use['lat']), 7)
            dawn_clean_df['Longitude [deg]'] = np.round(list(df_use['lon']), 7)

            if first_file:
                dawn_clean_df.to_csv(dawn_final_name, index = False)
                first_file = False
            else:
                df_all = pd.read_csv(dawn_final_name)
                df_total = pd.concat([df_all, dawn_clean_df], ignore_index = True)  #concatenates fields with same heading
                df_total.to_csv(dawn_final_name, index = False)


In [9]:
#plot up the available, good DAWN data for the given time range
df_all = pd.read_csv(dawn_final_name)

dawn_fig, dawn_axs = plt.subplots(nrows=1, ncols=2, figsize = (35,20))
dawn_fig.subplots_adjust(wspace=0.3)
dawn_x_ax = pd.to_datetime(df_all['Time [UTC]'])  #convert to datetime objects in order for set_major_formatter(mpl.dates.DateFormatter("%H:%M") to work
dawn_y_ax = df_all['Height [m]']
dawn_axs[0].scatter(dawn_x_ax, dawn_y_ax, s=15, c='k')
dawn_axs[0].tick_params(axis='x', rotation = 50)
dawn_axs[0].tick_params(labelsize=18)
dawn_axs[0].grid(True)
dawn_axs[0].set_xlabel('Time [UTC]', fontsize=30)
dawn_axs[0].set_ylabel('Height [m]', fontsize=30)
dawn_axs[0].set_title('DAWN Availability', fontsize=40)
dawn_axs[0].xaxis.set_major_formatter(mpl.dates.DateFormatter("%H:%M"))
dawn_axs[0].xaxis.set_major_locator(ticker.MaxNLocator(10))
#dawn_axs[0].gcf().set_size_inches(10,13)

#plot up same figure but with wind barbs instead of dots
dawn_u = df_all['U Comp of Wind [m/s]']
dawn_v = df_all['V Comp of Wind [m/s]']
dawn_axs[1].barbs(dawn_x_ax, dawn_y_ax, dawn_u, dawn_v, fill_empty = True, pivot='middle', sizes=dict(emptybarb=0.075), barbcolor = 'k')
#add "np.sqrt(dawn_u**2 + dawn_v**2)" to above line to color code barbs by speed
dawn_axs[1].tick_params(axis='x', rotation = 50)
dawn_axs[1].tick_params(labelsize=18)
dawn_axs[1].grid(True)
dawn_axs[1].set_xlabel('Time [UTC]', fontsize=30)
dawn_axs[1].set_ylabel('Height [m]', fontsize=30)
dawn_axs[1].set_title('DAWN 2-D Wind at Given Times and Heights', fontsize=40)
dawn_axs[1].xaxis.set_major_formatter(mpl.dates.DateFormatter("%H:%M"))
dawn_axs[1].xaxis.set_major_locator(ticker.MaxNLocator(10))
#dawn_axs[1].gcf().set_size_inches(20,25)

dawn_name = os.path.join(day_folder, "DAWN_avail_and_barbs_" + file_date + ".png")
plt.savefig(dawn_name, bbox_inches = 'tight')
plt.close()


In [10]:
#Ignore this cell if you're not Ben

#Calculate deep layer (500m - 8595.5m) shear for each qualifying DAWN profile using SHARPpy direct shear calculation method
#Results recorded in DAWN_Shear_calculations.csv

    #this step is done using DAWN_shear.py, so execute and see that script for how it's done
